In [1]:
pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/9y/gql5nk6n4fj3l56qdx4018n40000gn/T/pip-install-4hzu3h36/pytorch_52be7790a6a74709ad6aa2723598efde/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
Note: you may 

In [2]:
import pandas as pd
import numpy as np
import random as rand
import tqdm
import networkx as nx
import pickle as pkl
import gower
import matplotlib.pyplot as plt
import methods
from methods import topK, findGroups, prototype, ranking, getAttributes, Q, Q2, Discovery

/var/folders/9y/gql5nk6n4fj3l56qdx4018n40000gn/T/ipykernel_12202/53691989.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
from torch_geometric.datasets import Twitch

# Load the Tolokers dataset
dataset = Twitch(root='./data/Twitch', name='PT')

# Access the first graph in the dataset
data = dataset[0]

Processing...
Done!


In [9]:
dataset = Twitch(root='./data/Twitch', name='PT')

# Custom wrapper to simulate GraphPropPredDataset structure
class CustomGraphDataset:
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        # Extract a graph from the dataset
        graph_data = self.dataset[idx]
        
        # Convert to the desired dictionary format similar to GraphPropPredDataset
        graph = {
            'edge_index': graph_data.edge_index,  # Edge index
            'num_nodes': graph_data.num_nodes,  # Number of nodes
            'node_feat': graph_data.x  # Node features (if available)
        }
        
        label = graph_data.y
        return graph, label

    def __len__(self):
        return len(self.dataset)

# Instantiate the custom dataset
custom_dataset = CustomGraphDataset(dataset)
graph, label = custom_dataset[0]

# Access edge index, number of nodes, and node features
edge_index = graph['edge_index']
num_nodes = graph['num_nodes']

In [10]:
edge_index = data.edge_index 
num_nodes = data.num_nodes 
G = nx.Graph()

G.add_nodes_from(range(num_nodes))

edges = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))  # Convert tensor to list of tuples
G.add_edges_from(edges)

In [11]:
label

tensor([0, 0, 0,  ..., 1, 0, 1])

In [12]:
attributes = graph['node_feat']
lu = pd.DataFrame(attributes)
lu['target'] = label == 1 # use the label binary explicit language as target variable?
lu.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-0.213545,5.203393,-0.160547,-0.217247,-0.212379,-0.203340,-0.538649,-0.411488,-0.150398,-0.165370,...,-0.239307,-0.214107,-0.355245,-0.149007,-0.213861,-0.375776,1.312829,-0.255846,-0.183876,False
1,-0.236669,-0.230716,-0.160547,-0.130489,-0.212379,-0.203848,-0.538649,-0.411488,-0.162248,-0.223381,...,4.099865,-0.214107,-0.355245,-0.175422,0.012436,-0.373406,1.484898,-0.255846,-0.183876,False
2,-0.188405,-0.230716,-0.151268,-0.184601,-0.211119,-0.009926,-0.538649,-0.411488,-0.162248,-0.223381,...,-0.202553,-0.214107,-0.355245,-0.115830,-0.149139,-0.375621,1.406220,-0.255846,-0.172436,False
3,-0.206178,-0.230716,-0.160547,-0.217247,-0.172571,-0.203848,-0.538649,-0.411488,-0.162248,-0.223381,...,-0.174710,-0.214107,-0.355245,-0.175370,-0.200265,2.671659,1.353870,-0.255846,-0.183876,False
4,-0.236669,-0.230716,-0.160547,-0.217247,-0.212379,-0.202109,-0.538649,-0.411488,-0.162248,-0.223381,...,-0.239307,-0.214107,-0.355245,-0.175422,-0.213861,-0.375776,1.338136,-0.252739,-0.183876,False


In [13]:
result = findGroups(G, 20, lu)
result # this will take loooong on my PC

100%|██████████| 1912/1912 [58:59<00:00,  1.85s/it] 


,rho,sigma,q,ranks,reference,subgroup
385,9,4,0.277778,"[(385, True), (982, False), (1476, False), (13...","[385, 982, 1476, 1320, 69, 1104, 257, 1543, 1135]","[385, 982, 1476, 1320]"
1378,26,13,0.217571,"[(1378, True), (416, False), (1433, False), (2...","[1378, 416, 1433, 290, 127, 977, 642, 1698, 73...","[1378, 416, 1433, 290, 127, 977, 642, 1698, 73..."
97,12,4,0.192450,"[(97, True), (290, False), (1280, True), (26, ...","[97, 290, 1280, 26, 1486, 1105, 1543, 1514, 58...","[97, 290, 1280, 26]"
403,103,14,0.172833,"[(403, True), (1476, False), (467, False), (12...","[403, 1476, 467, 1245, 1787, 1142, 1660, 134, ...","[403, 1476, 467, 1245, 1787, 1142, 1660, 134, ..."
1032,220,58,0.163533,"[(1032, False), (1676, True), (1146, True), (3...","[1032, 1676, 1146, 31, 447, 577, 1201, 1848, 1...","[1032, 1676, 1146, 31, 447, 577, 1201, 1848, 1..."
947,48,7,0.160254,"[(947, False), (1036, False), (1647, True), (2...","[947, 1036, 1647, 235, 1185, 523, 1706, 1029, ...","[947, 1036, 1647, 235, 1185, 523, 1706]"
49,17,10,0.157905,"[(49, True), (1315, False), (127, False), (157...","[49, 1315, 127, 1570, 555, 1614, 274, 547, 161...","[49, 1315, 127, 1570, 555, 1614, 274, 547, 161..."
804,354,105,0.156089,"[(804, False), (287, True), (838, False), (2, ...","[804, 287, 838, 2, 1525, 1388, 1643, 1477, 409...","[804, 287, 838, 2, 1525, 1388, 1643, 1477, 409..."
767,66,8,0.155615,"[(767, False), (933, True), (560, True), (1809...","[767, 933, 560, 1809, 992, 1361, 190, 769, 176...","[767, 933, 560, 1809, 992, 1361, 190, 769]"
1240,79,24,0.155237,"[(1240, True), (250, False), (36, False), (144...","[1240, 250, 36, 1443, 44, 151, 1664, 1136, 135...","[1240, 250, 36, 1443, 44, 151, 1664, 1136, 135..."
